In [6]:
import pandas as pd

df = pd.read_csv('Credit Default Dataset.csv')

In [ ]:
#make first row as header and drop it from dataframe
header = df.iloc[0]
df.columns = header
df.drop(0 , inplace = True)

In [ ]:
#reset index after dropping the first row to maintain consistency
df.reset_index(drop = True , inplace =True)

In [ ]:
#convert all column names to lowercase
df.columns =[x.lower() for x in df.columns]

In [ ]:
#convert columns to numeric where possible errors='ignore' to leave non-convertible columns unchanged
df = df.apply(pd.to_numeric , errors='ignore')

/var/folders/r9/1c8zkb_x4d5fhpc5lbn_jndw0000gn/T/ipykernel_45128/624206244.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric , errors='ignore')


In [ ]:
#rename target column for easier access
df.rename(columns={'default payment next month': 'y'}, inplace=True)

1 - explore and analyze dataset 

In [27]:
df.head()

,id,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,y
0,1,20000,Female,university,married,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,Yes
1,2,120000,Female,university,single,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,Yes
2,3,90000,Female,university,single,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,No
3,4,50000,Female,university,married,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,No
4,5,50000,Male,university,married,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,No


In [8]:
df.shape

(30001, 25)

In [ ]:
#check for null values in each column
df.isnull().sum()

id                            0
limit_bal                     0
sex                           0
education                     0
marriage                      0
age                           0
pay_0                         0
pay_2                         0
pay_3                         0
pay_4                         0
pay_5                         0
pay_6                         0
bill_amt1                     0
bill_amt2                     0
bill_amt3                     0
bill_amt4                     0
bill_amt5                     0
bill_amt6                     0
pay_amt1                      0
pay_amt2                      0
pay_amt3                      0
pay_amt4                      0
pay_amt5                      0
pay_amt6                      0
default payment next month    0
dtype: int64

In [ ]:
#check for duplicate rows
df.duplicated().sum()

np.int64(0)

In [ ]:
#handle invalid values in 'education' and 'marriage' columns
df['education'] = df['education'].apply(lambda x: "others" if x in ["0", "5", "6"] else x)
df['marriage']  = df['marriage'].apply(lambda x: "others" if x == "0" else x)


In [ ]:
#explore categorical columns
for col in df.columns :
    if df[col].dtype == 'object':
        print(f'Column: {col} ')
        print(df[col].value_counts())
        print('-----------------------')

Column: sex 
sex
Female    18112
Male      11888
Name: count, dtype: int64
-----------------------
Column: education 
education
university         14030
graduate school    10585
high school         4917
others               468
Name: count, dtype: int64
-----------------------
Column: marriage 
marriage
single     15964
married    13659
others       377
Name: count, dtype: int64
-----------------------
Column: y 
y
No     23364
Yes     6636
Name: count, dtype: int64
-----------------------
Column: EDUCATION 
EDUCATION
university         14030
graduate school    10585
high school         4917
5                    280
others               123
6                     51
0                     14
Name: count, dtype: int64
-----------------------


In [60]:
def remove_outliers(df):
    """
    Removes outliers from all numeric columns in a DataFrame using the IQR method.
    Outliers are dropped in-place.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame
    
    Returns:
    pd.DataFrame: DataFrame with outliers removed
    """
    import pandas as pd
    
    df_clean = df.copy()
    num_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns
    
    for col in num_cols:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        
        # Find index of outliers
        outlier_index = df_clean[(df_clean[col] < lower) | (df_clean[col] > upper)].index
        
        # Drop outliers
        df_clean = df_clean.drop(outlier_index)
        
    return df_clean

In [61]:
df.to_csv('cleaned_credit_data.csv')